In [4]:
import math
import random
from typing import List, Dict, Any, Tuple
import numpy as np

# ============================================================
# CONFIG
# ============================================================
INSTANCE_PATH = r"C:\MDU\Masters\Nature Inspired\Group Project\Group 5\gecco19-thief\src\main\resources\a280-n279.txt"

RNG_SEED = 0
random.seed(RNG_SEED)
np.random.seed(RNG_SEED)

# Candidate list size
CAND_K = 3000

# Multi-start + ILS parameters
NUM_STARTS = 30           # how many different initial tours
ILS_RESTARTS = 200         # double-bridge kicks per best tour basin

# Local search parameters
TWO_OPT_MAX_ITERS = 10000          # per LS call
THREE_OPT_TRIES_PER_LS = 20000    # random 3-opt attempts per LS call

VERBOSE = True


# ============================================================
# TTP PARSER (simple, as in your LKH-lite)
# ============================================================
def parse_ttp(path: str) -> Dict[str, Any]:
    with open(path, "r") as f:
        lines = [line.strip() for line in f if line.strip()]

    dim = num_items = capacity = v_min = v_max = None
    edge_type = None

    i = 0
    while i < len(lines):
        L = lines[i]
        if L.startswith("DIMENSION"):
            dim = int(L.split(":")[1])
        elif L.startswith("NUMBER OF ITEMS"):
            num_items = int(L.split(":")[1])
        elif L.startswith("CAPACITY OF KNAPSACK"):
            capacity = float(L.split(":")[1])
        elif L.startswith("MIN SPEED"):
            v_min = float(L.split(":")[1])
        elif L.startswith("MAX SPEED"):
            v_max = float(L.split(":")[1])
        elif L.startswith("EDGE_WEIGHT_TYPE"):
            edge_type = L.split(":")[1].strip()
        elif L.startswith("NODE_COORD_SECTION"):
            break
        i += 1

    if dim is None:
        raise RuntimeError("DIMENSION not found in TTP file.")
    if edge_type is None:
        raise RuntimeError("EDGE_WEIGHT_TYPE not found in TTP file.")

    i += 1
    coords = np.zeros((dim, 2))
    for k in range(dim):
        idx, x, y = lines[i + k].split()
        coords[int(idx) - 1] = [float(x), float(y)]
    i += dim

    # Items (not needed for pure TSP, but we parse anyway)
    while i < len(lines) and "ITEMS SECTION" not in lines[i]:
        i += 1
    i += 1

    profits = np.zeros(num_items)
    weights = np.zeros(num_items)
    city = np.zeros(num_items, int)
    for j in range(num_items):
        p = lines[i + j].split()
        profits[j] = float(p[1])
        weights[j] = float(p[2])
        city[j] = int(p[3]) - 1

    return dict(
        dim=dim,
        coords=coords,
        items_profit=profits,
        items_weight=weights,
        items_city=city,
        capacity=capacity,
        v_min=v_min,
        v_max=v_max,
        edge_type=edge_type,
    )


# ============================================================
# DISTANCES + CANDIDATE LISTS
# ============================================================
def build_distance_matrix(coords: np.ndarray, edge: str):
    """
    TSPLIB-style distances.

    - EUC_2D:  int( sqrt(dx^2 + dy^2) + 0.5 )
    - CEIL_2D: int( ceil( sqrt(dx^2 + dy^2) ) )
    - else:    raw Euclidean as float
    """
    n = len(coords)
    edge = edge.upper() if edge else ""

    dist = [[0] * n for _ in range(n)]
    for i in range(n):
        xi, yi = coords[i]
        for j in range(i + 1, n):
            xj, yj = coords[j]
            dx = xi - xj
            dy = yi - yj
            dij = math.sqrt(dx * dx + dy * dy)

            if edge.startswith("EUC_2D"):
                dij = int(dij + 0.5)
            elif edge.startswith("CEIL_2D"):
                dij = int(math.ceil(dij))
            # else leave float

            dist[i][j] = dist[j][i] = dij

    return dist


def build_candidate_lists(dist, k: int):
    n = len(dist)
    C = []
    for i in range(n):
        nbr = sorted([(dist[i][j], j) for j in range(n) if j != i])[:k]
        C.append([j for _, j in nbr])
    return C


# ============================================================
# BASIC UTILITIES
# ============================================================
def tour_length(tour: List[int], dist) -> float:
    return sum(dist[tour[i]][tour[(i + 1) % len(tour)]] for i in range(len(tour)))


def check_permutation(tour: List[int], n: int):
    assert len(tour) == n, f"tour length {len(tour)} != {n}"
    s = set(tour)
    assert s == set(range(n)), f"tour not permutation of 0..{n-1}"


def ensure_valid(tour: List[int], n: int) -> List[int]:
    """Repair tour to a valid permutation, starting at 0."""
    seen = set()
    cleaned = []
    for c in tour:
        if 0 <= c < n and c not in seen:
            cleaned.append(c)
            seen.add(c)
    for c in range(n):
        if c not in seen:
            cleaned.append(c)
            seen.add(c)
    if 0 in cleaned:
        k = cleaned.index(0)
        cleaned = cleaned[k:] + cleaned[:k]
    else:
        cleaned = [0] + [c for c in range(1, n)]
    return cleaned[:n]


def nearest_neighbor_tour(dist, start: int = 0) -> List[int]:
    n = len(dist)
    unv = set(range(n))
    unv.remove(start)
    t = [start]
    cur = start
    while unv:
        nxt = min(unv, key=lambda j: dist[cur][j])
        t.append(nxt)
        unv.remove(nxt)
        cur = nxt
    return t


def random_greedy_tour(dist, cand, alpha: float = 0.3) -> List[int]:
    """
    Randomised greedy: at each step choose among a few nearest neighbours
    (RCL of size ~alpha * |cand[i]|).
    """
    n = len(dist)
    start = 0
    unv = set(range(n))
    unv.remove(start)
    t = [start]
    cur = start

    while unv:
        neigh = [j for j in cand[cur] if j in unv]
        if not neigh:
            # fallback: any remaining
            neigh = list(unv)
        neigh = sorted(neigh, key=lambda j: dist[cur][j])
        rcl_size = max(1, int(len(neigh) * alpha))
        rcl = neigh[:rcl_size]
        nxt = random.choice(rcl)
        t.append(nxt)
        unv.remove(nxt)
        cur = nxt

    return t


# ============================================================
# 2-OPT LOCAL SEARCH (candidate-guided)
# ============================================================
def two_opt_cand(tour: List[int], dist, cand, max_iter: int = 1000, label: str = "") -> Tuple[List[int], float]:
    n = len(tour)
    pos = [0] * n
    for i, c in enumerate(tour):
        pos[c] = i

    best_len = tour_length(tour, dist)
    if VERBOSE:
        print(f"[2-OPT{label}] start={best_len:.2f}")

    it = 0
    improved = True
    while improved and it < max_iter:
        improved = False
        it += 1

        for i in range(n):
            a = tour[i]
            b = tour[(i + 1) % n]

            for c in cand[a]:
                j = pos[c]
                if j <= i + 1 or j >= n - 1:
                    continue
                d = tour[(j + 1) % n]

                old = dist[a][b] + dist[c][d]
                new = dist[a][c] + dist[b][d]

                if new < old:
                    # apply 2-opt reversal
                    if i + 1 <= j:
                        tour[i + 1 : j + 1] = reversed(tour[i + 1 : j + 1])
                    else:
                        continue  # shouldn't happen with j>i+1

                    # update pos
                    for k, x in enumerate(tour):
                        pos[x] = k

                    best_len = tour_length(tour, dist)
                    if VERBOSE and it % 50 == 0:
                        print(f"[2-OPT{label}] it={it} len={best_len:.2f}")
                    improved = True
                    break

            if improved:
                break

    if VERBOSE:
        print(f"[2-OPT{label}] final={best_len:.2f}")
    return tour, best_len


# ============================================================
# RANDOM 3-OPT (subset of moves, sampled)
# ============================================================
def try_three_opt_move(
    tour: List[int],
    dist,
    i: int,
    j: int,
    k: int,
) -> Tuple[bool, List[int], float]:
    """
    Try a subset of 3-opt reconnections on segments:
    [0..i], [i+1..j], [j+1..k], [k+1..end]
    Returns (improved?, new_tour, new_length)
    """
    n = len(tour)
    a, b = tour[i], tour[(i + 1) % n]
    c, d = tour[j], tour[(j + 1) % n]
    e, f = tour[k], tour[(k + 1) % n]

    base = dist[a][b] + dist[c][d] + dist[e][f]

    # Pre-split
    s1 = tour[: i + 1]
    s2 = tour[i + 1 : j + 1]
    s3 = tour[j + 1 : k + 1]
    s4 = tour[k + 1 :]

    best_delta = 0.0
    best_tour = None

    # Move type 1: reverse s2
    cand1 = s1 + list(reversed(s2)) + s3 + s4
    d1 = (
        dist[a][tour[j]]
        + dist[tour[i + 1]][d]
        + dist[e][f]
    )
    delta1 = d1 - base
    if delta1 < best_delta:
        best_delta = delta1
        best_tour = cand1

    # Move type 2: reverse s3
    cand2 = s1 + s2 + list(reversed(s3)) + s4
    d2 = (
        dist[a][b]
        + dist[c][tour[k]]
        + dist[tour[j + 1]][f]
    )
    delta2 = d2 - base
    if delta2 < best_delta:
        best_delta = delta2
        best_tour = cand2

    # Move type 3: reverse s2 and s3
    cand3 = s1 + list(reversed(s2)) + list(reversed(s3)) + s4
    d3 = (
        dist[a][tour[j]]
        + dist[tour[i + 1]][tour[k]]
        + dist[tour[j + 1]][f]
    )
    delta3 = d3 - base
    if delta3 < best_delta:
        best_delta = delta3
        best_tour = cand3

    # Move type 4: swap s2 and s3 (no reversals)
    cand4 = s1 + s3 + s2 + s4
    d4 = (
        dist[a][c]
        + dist[e][b]
        + dist[tour[j]][f]
    )
    delta4 = d4 - base
    if delta4 < best_delta:
        best_delta = delta4
        best_tour = cand4

    if best_tour is not None and best_delta < 0:
        new_len = tour_length(best_tour, dist)
        return True, best_tour, new_len
    else:
        return False, tour, tour_length(tour, dist)


def three_opt_random(
    tour: List[int],
    dist,
    cand,
    tries: int = 10000,
    label: str = "",
) -> Tuple[List[int], float]:
    """
    Random 3-opt: sample triples (i,j,k) with candidate guidance.
    """
    n = len(tour)
    best_len = tour_length(tour, dist)
    if VERBOSE:
        print(f"[3-OPT{label}] start={best_len:.2f}")

    pos = [0] * n
    for idx, c in enumerate(tour):
        pos[c] = idx

    improved_any = False

    for t in range(tries):
        # pick random anchor city
        i = random.randint(0, n - 1)
        a = tour[i]

        # pick neighbor candidate for a
        cand_neighbors = cand[a]
        if not cand_neighbors:
            continue
        c1 = random.choice(cand_neighbors)
        j = pos[c1]

        # maintain ordering i < j < k in index space by rotating if needed
        if j == i or (j + 1) % n == i:
            continue

        # pick second anchor from neighbors of city at j
        bcity = tour[j]
        cand_neighbors2 = cand[bcity]
        if not cand_neighbors2:
            continue
        c2 = random.choice(cand_neighbors2)
        k = pos[c2]

        # ensure distinct, and choose ordering in index space
        idxs = sorted([i, j, k])
        i2, j2, k2 = idxs[0], idxs[1], idxs[2]
        if i2 == j2 or j2 == k2:
            continue

        changed, new_tour, new_len = try_three_opt_move(tour, dist, i2, j2, k2)
        if changed and new_len < best_len:
            tour = new_tour
            best_len = new_len
            improved_any = True

            # update pos after accepted move
            pos = [0] * n
            for idx, c in enumerate(tour):
                pos[c] = idx

            if VERBOSE and (t + 1) % 1000 == 0:
                print(f"[3-OPT{label}] t={t+1} len={best_len:.2f}")

    if VERBOSE:
        print(f"[3-OPT{label}] final={best_len:.2f} (improved={improved_any})")
    return tour, best_len


# ============================================================
# DOUBLE-BRIDGE KICK (ILS PERTURBATION)
# ============================================================
def double_bridge_move(tour: List[int]) -> List[int]:
    n = len(tour)
    if n < 8:
        return tour[:]
    cuts = sorted(random.sample(range(1, n), 4))
    a, b, c, d = cuts
    s1 = tour[0:a]
    s2 = tour[a:b]
    s3 = tour[b:c]
    s4 = tour[c:d]
    s5 = tour[d:]
    return s1 + s3 + s2 + s4 + s5


# ============================================================
# LK-FLAVOURED MULTI-START + ILS
# ============================================================
def lk_style_tsp_from_ttp(
    data: Dict[str, Any],
    cand_k: int = CAND_K,
    num_starts: int = NUM_STARTS,
    ils_restarts: int = ILS_RESTARTS,
) -> Tuple[List[int], float, Any]:
    coords = data["coords"]
    edge = data["edge_type"]
    n = len(coords)

    print(f"[INFO] EDGE_WEIGHT_TYPE = {edge}")
    print("[DIST] Building distance matrix...")
    dist = build_distance_matrix(coords, edge)

    print(f"[CAND] Building candidate lists (k={cand_k})...")
    cand = build_candidate_lists(dist, cand_k)

    # --------------------------------------------------------
    # INITIAL TOURS (multi-start)
    # --------------------------------------------------------
    best_global_tour = None
    best_global_len = float("inf")

    for s in range(num_starts):
        print(f"\n[START {s+1}/{num_starts}] constructing initial tour...")

        if s == 0:
            base_tour = nearest_neighbor_tour(dist, start=0)
            base_tour = ensure_valid(base_tour, n)
        else:
            base_tour = random_greedy_tour(dist, cand, alpha=0.3)
            base_tour = ensure_valid(base_tour, n)

        base_len = tour_length(base_tour, dist)
        print(f"[BASE {s+1}] initial length={base_len:.2f}")

        # First pass: strong 2-opt
        tour_ls, len_ls = two_opt_cand(
            base_tour,
            dist,
            cand,
            max_iter=TWO_OPT_MAX_ITERS,
            label=f" S{s+1}"
        )

        # Then random 3-opt to deepen basin
        tour_ls, len_ls = three_opt_random(
            tour_ls,
            dist,
            cand,
            tries=THREE_OPT_TRIES_PER_LS,
            label=f" S{s+1}"
        )

        print(f"[BASIN {s+1}] local min length={len_ls:.2f}")

        # ----------------------------------------------------
        # ILS with double-bridge + local search again
        # ----------------------------------------------------
        best_basin_tour = tour_ls[:]
        best_basin_len = len_ls

        for r in range(ils_restarts):
            print(f"[ILS S{s+1}] restart {r+1}/{ils_restarts}")
            kicked = double_bridge_move(best_basin_tour)
            kicked = ensure_valid(kicked, n)

            k_len = tour_length(kicked, dist)
            if VERBOSE:
                print(f"[ILS S{s+1}] kicked length={k_len:.2f}")

            # local search again from kicked point
            kicked, k_len = two_opt_cand(
                kicked,
                dist,
                cand,
                max_iter=TWO_OPT_MAX_ITERS // 2,
                label=f" S{s+1}R{r+1}"
            )
            kicked, k_len = three_opt_random(
                kicked,
                dist,
                cand,
                tries=THREE_OPT_TRIES_PER_LS // 2,
                label=f" S{s+1}R{r+1}"
            )

            if k_len < best_basin_len:
                print(f"[ILS S{s+1}] improved basin {best_basin_len:.2f} -> {k_len:.2f}")
                best_basin_len = k_len
                best_basin_tour = kicked[:]
            else:
                print(f"[ILS S{s+1}] no improvement ({k_len:.2f} >= {best_basin_len:.2f})")

        # Update global best
        if best_basin_len < best_global_len:
            print(f"[GLOBAL] improved {best_global_len:.2f} -> {best_basin_len:.2f}")
            best_global_len = best_basin_len
            best_global_tour = best_basin_tour[:]

    # Final sanity checks
    check_permutation(best_global_tour, n)
    true_len = tour_length(best_global_tour, dist)
    print(f"\n[RESULT] best length={best_global_len:.2f}")
    print(f"[CHECK] tour_length(best)={true_len:.2f}")

    return best_global_tour, best_global_len, dist


# ============================================================
# OPTIONAL: TTP EMPTY-PACK TRAVEL TIME CHECK
# ============================================================
def ttp_time_empty(tour: List[int], dist, data: Dict[str, Any]) -> float:
    """
    TTP travel time with no items picked, using TTP speed function.
    """
    weights = data["items_weight"]
    cities = data["items_city"]
    capacity = data["capacity"]
    v_min = data["v_min"]
    v_max = data["v_max"]

    picked = np.zeros_like(weights, dtype=int)

    current_w = 0.0
    n = len(tour)
    slope = (v_max - v_min) / capacity
    total_time = 0.0

    for i in range(n):
        c = tour[i]
        nxt = tour[(i + 1) % n]

        # no items taken, but we keep structure if you later want to reuse this
        for item_idx in np.where(cities == c)[0]:
            if picked[item_idx] == 1:
                current_w += weights[item_idx]

        speed = v_max - slope * current_w
        if speed < 1e-9:
            return float("inf")

        total_time += dist[c][nxt] / speed

    return total_time


# ============================================================
# MAIN ENTRYPOINT
# ============================================================
def main():
    print(f"[LOAD] Reading instance from: {INSTANCE_PATH}")
    data = parse_ttp(INSTANCE_PATH)
    print(f"[LOAD] Parsed {data['dim']} cities, {len(data['items_profit'])} items.")

    best_tour, best_len, dist = lk_style_tsp_from_ttp(
        data,
        cand_k=CAND_K,
        num_starts=NUM_STARTS,
        ils_restarts=ILS_RESTARTS,
    )

    empty_time = ttp_time_empty(best_tour, dist, data)
    print("\n========== FINAL SUMMARY ==========")
    print(f"TSP best length        : {best_len:.2f}")
    print(f"TTP empty-pack time    : {empty_time:.2f}")
    print("===================================")

    return best_tour, best_len, empty_time


if __name__ == "__main__":
    main()


[LOAD] Reading instance from: C:\MDU\Masters\Nature Inspired\Group Project\Group 5\gecco19-thief\src\main\resources\a280-n279.txt
[LOAD] Parsed 280 cities, 279 items.
[INFO] EDGE_WEIGHT_TYPE = CEIL_2D
[DIST] Building distance matrix...
[CAND] Building candidate lists (k=3000)...

[START 1/30] constructing initial tour...
[BASE 1] initial length=3160.00
[2-OPT S1] start=3160.00
[2-OPT S1] final=2821.00
[3-OPT S1] start=2821.00
[3-OPT S1] final=2821.00 (improved=False)
[BASIN 1] local min length=2821.00
[ILS S1] restart 1/200
[ILS S1] kicked length=3056.00
[2-OPT S1R1] start=3056.00
[2-OPT S1R1] final=2850.00
[3-OPT S1R1] start=2850.00
[3-OPT S1R1] final=2850.00 (improved=False)
[ILS S1] no improvement (2850.00 >= 2821.00)
[ILS S1] restart 2/200
[ILS S1] kicked length=2890.00
[2-OPT S1R2] start=2890.00
[2-OPT S1R2] final=2822.00
[3-OPT S1R2] start=2822.00
[3-OPT S1R2] final=2822.00 (improved=False)
[ILS S1] no improvement (2822.00 >= 2821.00)
[ILS S1] restart 3/200
[ILS S1] kicked length